# Crusher chest notebook

This is a simulation. The first code cell is all of the functions. The second code cells is where you enter your information to be calculated.

## Functions

In [1]:
#===========#
#  imports  #
#===========#

import pandas as pd
from random import choice

#=============#
#  functions  #
#=============#

#== how many total medals do you have ==#
def medals(star, medal_count, bonus=True):
    base_medals = {1: 10,
                   2: 30,
                   3: 80,
                   4: 280,
                   5: 880,
                   6: 2380}
    
    if bonus == True and star < 6:
        gift = 400
    else:
        gift = 0
        
    return base_medals[star] + medal_count + gift

#== simulation to determine average crusher yields ==#
def simulate(runs):
    # contents of a single crusher chest
    chest = ['crusher','epic','rare1','rare2','rare3','rare4']

    # rolls at each teir.  First value is guaranteed crusher medals
    tiers = {1 : [10,80],
             2 : [30,100,20],
             3 : [50,100,80],
             4 : [80,100,100,80],
             5 : [120,100,100,100,100]
             }

    # An empty collection of all Tier 1 crusher chest medals
    results = {1 : {'crusher' : 0,
                    'epic' : 0,
                    'rare1' : 0,
                    'rare2' : 0,
                    'rare3' : 0,
                    'rare4' : 0
                   }
              }
    
    # making empty collections for Tiers 2 - 5
    for key in range(2,6):
        results[key] = results[1].copy()

    # for each run, buy one chest of each tier
    for n in range(runs):
        for tier in range(1,6):
            currentChest = chest.copy() # copy of chest to edit per run
            results[tier]['crusher'] += tiers[tier][0] # add 0th element as guaranteed medals
            for pull in tiers[tier][1:]:
                roll = choice(currentChest) # random choice
                results[tier][roll] += pull # update the random choice
                currentChest.remove(roll) # can only pull hero once per chest tier

    # average results for each  chest tier
    for tier in range(1,6):
        for rarity in chest:
            results[tier][rarity] = results[tier][rarity]/runs
            
            # add in results of previous tier for tiers > 1
            if tier > 1:
                results[tier][rarity] += results[tier-1][rarity]
                
            results[tier][rarity] = round(results[tier][rarity],2)

    # create average yield list
    crushers = []
    for i in range(1,6):
        crushers.append(results[i]['crusher'])
    
    return crushers

#== creating a table to be viewed ==#
def create_table(avg_crusher, current_medals):
    crusher_dict = {'avg crusher' : avg_crusher,
                    'total flooz': [120,360,810,1710,3510],
                   }

    df = pd.DataFrame(crusher_dict,index=range(1,6))
    df.index.name = 'Tier'

    medals_remaining = 4880-current_medals

    cpf = df['avg crusher'] / df['total flooz']
    total_flooz = medals_remaining / cpf
    df['flooz to 7s'] = round(total_flooz,2)

    days = df['flooz to 7s'] / df['total flooz'] * 24
    df['days to 7s'] = round(days,2)

    months = df['days to 7s'] / 30
    df['months to 7s'] = round(months,2)

    cycles = df['days to 7s'] / 24
    df['cycles to 7s'] = round(cycles,2)

    return df

#== there is no way I am running each of the above... ==#
def calculate(star, medal_count, bonus=True, runs=10_000):
    my_medals = medals(star,medal_count,bonus)
    avg_crusher = simulate(runs)
    table = create_table(avg_crusher,my_medals)
    return table

## Calculator

In [2]:
#== my Sorrow info ==#
star = 6
current_medals = 1382
bonus = True
runs = 20_000

table = calculate(star, current_medals, bonus, runs)
display(table)

,avg crusher,total flooz,flooz to 7s,days to 7s,months to 7s,cycles to 7s
Tier,,,,,,
1,23.24,120,5772.81,1154.56,38.49,48.11
2,73.12,360,5504.38,366.96,12.23,15.29
3,152.73,810,5929.29,175.68,5.86,7.32
4,279.44,1710,6841.47,96.02,3.20,4.00
5,465.48,3510,8430.39,57.64,1.92,2.40
